## Big Data Analytics Final Project

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from math import sqrt

In [498]:
import findspark
findspark.init()
from pyspark.mllib.clustering import KMeans
from pyspark.sql import SparkSession

from pyspark.sql.functions import unix_timestamp
import pyspark.sql.functions as f

In [499]:
sc = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

In [509]:
df_csv = sc.read.csv('cases_by_status_and_phu.csv', header=True, inferSchema=True)

In [262]:
df_csv.show(20,False)

+---------+---------------------------------------+-------+------------+--------------+------+
|FILE_DATE|PHU_NAME                               |PHU_NUM|ACTIVE_CASES|RESOLVED_CASES|DEATHS|
+---------+---------------------------------------+-------+------------+--------------+------+
|20200410 |HALIBURTON, KAWARTHA, PINE RIDGE       |2235   |23          |69            |18    |
|20200410 |HALTON REGION                          |2236   |112         |137           |9     |
|20200410 |HASTINGS & PRINCE EDWARD COUNTIES      |2238   |12          |11            |1     |
|20200410 |HURON PERTH                            |5183   |20          |8             |1     |
|20200410 |KINGSTON, FRONTENAC, LENNOX & ADDINGTON|2241   |13          |39            |0     |
|20200410 |LAMBTON COUNTY                         |2242   |65          |26            |9     |
|20200410 |LEEDS, GRENVILLE AND LANARK DISTRICT   |2243   |102         |36            |8     |
|20200410 |MIDDLESEX-LONDON                       

In [406]:
df_json = sc.read.option("multiline","true") \
      .json("phu_locations.json")

In [407]:
df_json.printSchema()

root
 |-- PHU: string (nullable = true)
 |-- PHU_fr: string (nullable = true)
 |-- accessible: string (nullable = true)
 |-- active: string (nullable = true)
 |-- additional_information: string (nullable = true)
 |-- additional_information_fr: string (nullable = true)
 |-- address: string (nullable = true)
 |-- address_fr: string (nullable = true)
 |-- after_hours: string (nullable = true)
 |-- age_threshold: string (nullable = true)
 |-- appointments: string (nullable = true)
 |-- asl_interpretation: string (nullable = true)
 |-- assessment_centre: string (nullable = true)
 |-- children_under_2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- community_lab: string (nullable = true)
 |-- drive_through: string (nullable = true)
 |-- email: string (nullable = true)
 |-- first_nations: string (nullable = true)
 |-- free_parking: string (nullable = true)
 |-- french_language_services: string (nullable = true)
 |-- friday: string (nullable = true)
 |-- general_population: 

In [409]:
df_json.show(20,False)

+--------------------------------------+------------------------------------------------------+----------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------+----------------------------+--------------------------+-----------+-------------+------------+------------------+-----------------+----------------+-------------+-------------+-------------+-----+-------------+------------+------------------------+------------+------------------+---------+-----------+----------------------------

In [410]:
df_json.select("location_name").show(20,False)

+---------------------------------------------------------------+
|location_name                                                  |
+---------------------------------------------------------------+
|Kirkland and District Hospital                                 |
|South Georgian Bay Regional Assessment Centre                  |
|Midland Assessment Centre                                      |
|Milton District Hospital                                       |
|Oakville Trafalgar Memorial Hospital                           |
|Georgetown Hospital                                            |
|Headwaters Health Care Centre                                  |
|MacKenzie Health                                               |
|Markham Stouffville Hospital                                   |
|Bracebridge Rotary Centre                                      |
|Huntsville District Memorial Hospital                          |
|Orillia Soldiers' Memorial Hospital                            |
|Barrie Co

In [411]:
df_json.printSchema()

root
 |-- PHU: string (nullable = true)
 |-- PHU_fr: string (nullable = true)
 |-- accessible: string (nullable = true)
 |-- active: string (nullable = true)
 |-- additional_information: string (nullable = true)
 |-- additional_information_fr: string (nullable = true)
 |-- address: string (nullable = true)
 |-- address_fr: string (nullable = true)
 |-- after_hours: string (nullable = true)
 |-- age_threshold: string (nullable = true)
 |-- appointments: string (nullable = true)
 |-- asl_interpretation: string (nullable = true)
 |-- assessment_centre: string (nullable = true)
 |-- children_under_2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- community_lab: string (nullable = true)
 |-- drive_through: string (nullable = true)
 |-- email: string (nullable = true)
 |-- first_nations: string (nullable = true)
 |-- free_parking: string (nullable = true)
 |-- french_language_services: string (nullable = true)
 |-- friday: string (nullable = true)
 |-- general_population: 

In [412]:
df_json.drop(col("PHU_fr"))
df_json.drop(col("additional_information_fr"))
df_json.drop(col("address_fr"))
df_json.drop(col("location_name_fr"))
df_json.drop(col("operated_by_fr"))

DataFrame[PHU: string, PHU_fr: string, accessible: string, active: string, additional_information: string, additional_information_fr: string, address: string, address_fr: string, after_hours: string, age_threshold: string, appointments: string, asl_interpretation: string, assessment_centre: string, children_under_2: string, city: string, community_lab: string, drive_through: string, email: string, first_nations: string, free_parking: string, french_language_services: string, friday: string, general_population: string, latitude: string, location_id: string, location_name: string, location_name_fr: string, longitude: string, monday: string, online_appointments: string, operated_by: string, paid_parking: string, pharmacy: string, phone: string, phone_appointments: string, phu_id: string, postal_code: string, province: string, public_transit: string, saturday: string, sunday: string, symptomatic: string, temporarily_closed: string, thursday: string, tuesday: string, walk_ins: string, websi

In [71]:

df_json.createOrReplaceTempView("locations")

# Question 1:

List all PHU locations which are allow online appointments. List should not contain columns in
French. All French columns have name ending with fr.


We already removed the columns ending in Fr. We check some of the values of the online_appointmnent in the data frame.

In [392]:
df_json.select("online_appointments").show(20,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------+
|online_appointments                                                                                                                                |
+---------------------------------------------------------------------------------------------------------------------------------------------------+
|null                                                                                                                                               |
|https://gbfht.ca/cac/                                                                                                                              |
|null                                                                                                                                               |
|https://www.haltonhealthcare.on.ca/covid-19-info/booking-a-test                                    

In [413]:
query='SELECT PHU, location_name as Location from locations where online_appointments IS NOT NULL'

In [414]:
df_json = sc.sql(query)
df_json.show(20,False)

+------------------------------------------------------+------------------------------------------------------------------------------------+
|PHU                                                   |Location                                                                            |
+------------------------------------------------------+------------------------------------------------------------------------------------+
|Simcoe Muskoka District Health Unit                   |South Georgian Bay Regional Assessment Centre                                       |
|Halton Region Health Department                       |Milton District Hospital                                                            |
|Halton Region Health Department                       |Oakville Trafalgar Memorial Hospital                                                |
|Halton Region Health Department                       |Georgetown Hospital                                                                 |
|York 

# Question 2:

List all the PHU locations allow children under 2 in Brampton location.

In [99]:
query='SELECT PHU,location_name as Location, City from locations where children_under_2=="Yes" and city=="Brampton"'


In [102]:
df_q2 = sc.sql(query)
df_q2.show(20,False)

+------------------+--------------------------------+--------+
|PHU               |Location                        |City    |
+------------------+--------------------------------+--------+
|Peel Public Health|10 Cottrelle Blvd - Dynacare    |Brampton|
|Peel Public Health|7700 Hurontario - Dynacare      |Brampton|
|Peel Public Health|9525 Mississauga Road - Dynacare|Brampton|
+------------------+--------------------------------+--------+



As we see, the PHUs in Brmapton that allow children under the age of two is very limited.

# Question 3:

 List all the PHU locations in Toronto has free parking with drive through.

In [121]:
query='SELECT PHU,location_name as Location, City from locations where city=="Toronto" and free_parking=="Yes" and drive_through=="Yes"'


In [122]:
df_q3 = sc.sql(query)
df_q3.show(20,False)

+---+--------+----+
|PHU|Location|City|
+---+--------+----+
+---+--------+----+



The query tells us there are no PHUs in toronto that offer free parking along with having a drive through.

# Question 4:

Find out number of PHUs are closed in each city on Friday

In [191]:
df_json.select("friday","PHU","City").show(20,False)

+------------+--------------------------------------+-------------+
|friday      |PHU                                   |City         |
+------------+--------------------------------------+-------------+
|09:00-13:00 |Timiskaming Health Unit               |Kirkland Lake|
|09:00-17:00 |Simcoe Muskoka District Health Unit   |Collingwood  |
|08:00-16:00 |Simcoe Muskoka District Health Unit   |Midland      |
|08:00-17:00 |Halton Region Health Department       |Milton       |
|08:00-20:00 |Halton Region Health Department       |Oakville     |
|null        |Halton Region Health Department       |Georgetown   |
|10:00-22:00 |Wellington-Dufferin-Guelph Health Unit|Orangeville  |
|10:00-21:30 |York Region Public Health Services    |Richmond Hill|
|09:00-19:00 |York Region Public Health Services    |Markham      |
|09:00-17:00 |Simcoe Muskoka District Health Unit   |Bracebridge  |
|09:00-17:00 |Simcoe Muskoka District Health Unit   |Huntsville   |
|11:00-18:30 |Simcoe Muskoka District Health Uni

In [192]:
query='SELECT COUNT(PHU), City from locations where friday is null group by City '


In [194]:
df_q4 = sc.sql(query)
df_q4.show(30,False)

+----------+-----------------+
|count(PHU)|City             |
+----------+-----------------+
|1         |Georgetown       |
|1         |Sarnia           |
|1         |Sunderland       |
|1         |Orangeville      |
|1         |Alexandria       |
|1         |Arnprior         |
|1         |Matheson         |
|1         |Campbellford     |
|1         |Maple            |
|1         |Renfrew          |
|1         |Harriston        |
|1         |Fort Albany      |
|1         |Malton           |
|1         |Innisfil         |
|5         |Toronto          |
|1         |Barry's Bay      |
|1         |Killaloe         |
|1         |Terrace Bay      |
|1         |Southampton      |
|1         |Beaverton        |
|1         |Smooth Rock Falls|
|1         |Windsor          |
|1         |Brampton         |
|1         |Douglas          |
|1         |East York        |
|1         |North York       |
+----------+-----------------+



The results of the query are  mostly similar with one visible outlier being Toronto, where five PHUs are closed on Friday. 

# Question 5:

Find out all the PHU locations which are open during 20:00 and 21:00 on Monday in Toronto city

In [203]:
df_json.filter(df_json.city=="Toronto").select("monday","PHU","City").show(20,False)

+-----------+----------------------------------+-------+
|monday     |PHU                               |City   |
+-----------+----------------------------------+-------+
|11:00-19:00|Toronto Public Health             |Toronto|
|00:00-00:00|Toronto Public Health             |Toronto|
|08:00-19:30|Toronto Public Health             |Toronto|
|13:00-18:30|York Region Public Health Services|Toronto|
|08:00-16:00|Toronto Public Health             |Toronto|
|08:00-20:00|Toronto Public Health             |Toronto|
|null       |Toronto Public Health             |Toronto|
|09:00-19:30|Toronto Public Health             |Toronto|
|08:00-20:00|Toronto Public Health             |Toronto|
|08:00-20:00|Toronto Public Health             |Toronto|
|08:00-18:00|Toronto Public Health             |Toronto|
|08:30-19:30|Toronto Public Health             |Toronto|
|08:00-00:00|Toronto Public Health             |Toronto|
|08:00-22:00|Toronto Public Health             |Toronto|
|08:00-00:00|Toronto Public Hea

In [196]:
query='SELECT PHU, location_name as Location, City from locations where monday Like "20:00%" or monday like "21:00%" and City=="Toronto" '


In [197]:
df_q5 = sc.sql(query)
df_q5.show(30,False)

+---+--------+----+
|PHU|Location|City|
+---+--------+----+
+---+--------+----+



As can be seen, there arent any PHUs in Toronto that operate in these specific times.

# Question 6:

List the hours of operation of all the PHU in Toronto. 

In [213]:
df_json.printSchema()

root
 |-- PHU: string (nullable = true)
 |-- PHU_fr: string (nullable = true)
 |-- accessible: string (nullable = true)
 |-- active: string (nullable = true)
 |-- additional_information: string (nullable = true)
 |-- additional_information_fr: string (nullable = true)
 |-- address: string (nullable = true)
 |-- address_fr: string (nullable = true)
 |-- after_hours: string (nullable = true)
 |-- age_threshold: string (nullable = true)
 |-- appointments: string (nullable = true)
 |-- asl_interpretation: string (nullable = true)
 |-- assessment_centre: string (nullable = true)
 |-- children_under_2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- community_lab: string (nullable = true)
 |-- drive_through: string (nullable = true)
 |-- email: string (nullable = true)
 |-- first_nations: string (nullable = true)
 |-- free_parking: string (nullable = true)
 |-- french_language_services: string (nullable = true)
 |-- friday: string (nullable = true)
 |-- general_population: 

In [209]:
query='SELECT PHU, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday from locations where City=="Toronto" '


In [212]:
df_q6 = sc.sql(query)
df_q6.show(20,False)

+----------------------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|PHU                               |Monday     |Tuesday    |Wednesday  |Thursday   |Friday     |Saturday   |Sunday     |
+----------------------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|Toronto Public Health             |11:00-19:00|11:00-19:00|11:00-19:00|11:00-19:00|11:00-19:00|08:00-16:00|08:00-16:00|
|Toronto Public Health             |00:00-00:00|00:00-00:00|00:00-00:00|00:00-00:00|00:00-00:00|00:00-00:00|00:00-00:00|
|Toronto Public Health             |08:00-19:30|08:00-19:30|08:00-19:30|08:00-19:30|08:00-19:30|08:00-19:30|08:00-19:30|
|York Region Public Health Services|13:00-18:30|13:00-18:30|13:00-18:30|null       |null       |null       |null       |
|Toronto Public Health             |08:00-16:00|08:00-16:00|08:00-16:00|08:00-16:00|08:00-16:00|           |           |
|Toronto Public Health          

# Question 7:

Count all the PHU are temporary closed in each city.

In [216]:
df_json.select("temporarily_closed").show(20,False)

+------------------+
|temporarily_closed|
+------------------+
|null              |
|null              |
|null              |
|null              |
|null              |
|null              |
|null              |
|null              |
|null              |
|Yes               |
|Yes               |
|null              |
|null              |
|null              |
|null              |
|null              |
|null              |
|null              |
|null              |
|null              |
+------------------+
only showing top 20 rows



In [225]:
query='SELECT Count(PHU), City from locations where temporarily_closed=="Yes" GROUP BY City '


In [226]:
df_q7 = sc.sql(query)
df_q7.show(20,False)

+----------+-----------+
|count(PHU)|City       |
+----------+-----------+
|1         |Cobden     |
|1         |Alexandria |
|1         |Welland    |
|1         |Goderich   |
|1         |Casselman  |
|1         |Oshawa     |
|2         |Toronto    |
|1         |Huntsville |
|1         |Kitchener  |
|2         |Windsor    |
|1         |Hamilton   |
|1         |Haileybury |
|1         |Petawawa   |
|1         |Douglas    |
|1         |Bracebridge|
+----------+-----------+



# Question 8:

phu_locations.json has phone field. Create two more columns. First field should contain only
area code and 2nd field should contain extension number.


In [228]:
df_json.printSchema()

root
 |-- PHU: string (nullable = true)
 |-- PHU_fr: string (nullable = true)
 |-- accessible: string (nullable = true)
 |-- active: string (nullable = true)
 |-- additional_information: string (nullable = true)
 |-- additional_information_fr: string (nullable = true)
 |-- address: string (nullable = true)
 |-- address_fr: string (nullable = true)
 |-- after_hours: string (nullable = true)
 |-- age_threshold: string (nullable = true)
 |-- appointments: string (nullable = true)
 |-- asl_interpretation: string (nullable = true)
 |-- assessment_centre: string (nullable = true)
 |-- children_under_2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- community_lab: string (nullable = true)
 |-- drive_through: string (nullable = true)
 |-- email: string (nullable = true)
 |-- first_nations: string (nullable = true)
 |-- free_parking: string (nullable = true)
 |-- french_language_services: string (nullable = true)
 |-- friday: string (nullable = true)
 |-- general_population: 

In [229]:
df_json.select("phone").show(20,False)

+----------------------+
|phone                 |
+----------------------+
|705-568-2127          |
|705-444-5885          |
|705-529-1025          |
|905-203-7963          |
|905-203-7963          |
|905-203-7963          |
|833-442-2001          |
|905-883-1212 ext. 2004|
|null                  |
|888-383-7009          |
|888-383-7009          |
|888-589-4109          |
|null                  |
|289-803-2199          |
|249-501-0383          |
|                      |
|null                  |
|                      |
|905-487-1249          |
|905-487-1249          |
+----------------------+
only showing top 20 rows



In [490]:
### String Split of the column in pyspark
from pyspark.sql.functions import split, col

In [259]:
df_phone=df_json.select("phone")

In [261]:
df_phone.withColumn("area_code", split(col("phone"), "-").getItem(0)).withColumn("extension", split(col("phone"), "ext.").getItem(1)).show(30,False)

+-----------------------+---------+---------+
|phone                  |area_code|extension|
+-----------------------+---------+---------+
|705-568-2127           |705      |null     |
|705-444-5885           |705      |null     |
|705-529-1025           |705      |null     |
|905-203-7963           |905      |null     |
|905-203-7963           |905      |null     |
|905-203-7963           |905      |null     |
|833-442-2001           |833      |null     |
|905-883-1212 ext. 2004 |905      | 2004    |
|null                   |null     |null     |
|888-383-7009           |888      |null     |
|888-383-7009           |888      |null     |
|888-589-4109           |888      |null     |
|null                   |null     |null     |
|289-803-2199           |289      |null     |
|249-501-0383           |249      |null     |
|                       |         |null     |
|null                   |null     |null     |
|                       |         |null     |
|905-487-1249           |905      

# Question 9:

Find out highest number of resolved cases of covid-19 in each PHU between April 2020 to Sep
2020

In [263]:
df_csv.printSchema()

root
 |-- FILE_DATE: integer (nullable = true)
 |-- PHU_NAME: string (nullable = true)
 |-- PHU_NUM: integer (nullable = true)
 |-- ACTIVE_CASES: integer (nullable = true)
 |-- RESOLVED_CASES: integer (nullable = true)
 |-- DEATHS: integer (nullable = true)



In [264]:
df_csv.show(30,False)

+---------+---------------------------------------+-------+------------+--------------+------+
|FILE_DATE|PHU_NAME                               |PHU_NUM|ACTIVE_CASES|RESOLVED_CASES|DEATHS|
+---------+---------------------------------------+-------+------------+--------------+------+
|20200410 |HALIBURTON, KAWARTHA, PINE RIDGE       |2235   |23          |69            |18    |
|20200410 |HALTON REGION                          |2236   |112         |137           |9     |
|20200410 |HASTINGS & PRINCE EDWARD COUNTIES      |2238   |12          |11            |1     |
|20200410 |HURON PERTH                            |5183   |20          |8             |1     |
|20200410 |KINGSTON, FRONTENAC, LENNOX & ADDINGTON|2241   |13          |39            |0     |
|20200410 |LAMBTON COUNTY                         |2242   |65          |26            |9     |
|20200410 |LEEDS, GRENVILLE AND LANARK DISTRICT   |2243   |102         |36            |8     |
|20200410 |MIDDLESEX-LONDON                       

In [321]:
df_csv.createOrReplaceTempView("cases")

In [280]:
query2='SELECT Max(ACTIVE_CASES), PHU_NAME as PHU from cases where (FILE_DATE BETWEEN "20200401"AND "20200930") group by PHU_NAME ORDER by MAX(ACTIVE_CASES) ASC'


In [281]:
df_q9 = sc.sql(query2)
df_q9.show(30,False)

+-----------------+---------------------------------------+
|max(ACTIVE_CASES)|PHU                                    |
+-----------------+---------------------------------------+
|6                |NORTH BAY PARRY SOUND DISTRICT         |
|7                |ALGOMA DISTRICT                        |
|7                |TIMISKAMING                            |
|9                |NORTHWESTERN                           |
|10               |RENFREW COUNTY AND DISTRICT            |
|14               |HASTINGS & PRINCE EDWARD COUNTIES      |
|21               |PORCUPINE                              |
|22               |SUDBURY AND DISTRICT                   |
|24               |BRANT COUNTY                           |
|26               |THUNDER BAY DISTRICT                   |
|27               |PETERBOROUGH COUNTY-CITY               |
|31               |HURON PERTH                            |
|35               |GREY BRUCE                             |
|35               |EASTERN ONTARIO      

# Question 10:

Which PHU has more resolved cases of covid-19 than active cases in year 2020.


In [293]:
df_csv.selectshow(20,False)


+---------+---------------------------------------+-------+------------+--------------+------+
|FILE_DATE|PHU_NAME                               |PHU_NUM|ACTIVE_CASES|RESOLVED_CASES|DEATHS|
+---------+---------------------------------------+-------+------------+--------------+------+
|20200410 |HALIBURTON, KAWARTHA, PINE RIDGE       |2235   |23          |69            |18    |
|20200410 |HALTON REGION                          |2236   |112         |137           |9     |
|20200410 |HASTINGS & PRINCE EDWARD COUNTIES      |2238   |12          |11            |1     |
|20200410 |HURON PERTH                            |5183   |20          |8             |1     |
|20200410 |KINGSTON, FRONTENAC, LENNOX & ADDINGTON|2241   |13          |39            |0     |
|20200410 |LAMBTON COUNTY                         |2242   |65          |26            |9     |
|20200410 |LEEDS, GRENVILLE AND LANARK DISTRICT   |2243   |102         |36            |8     |
|20200410 |MIDDLESEX-LONDON                       

In [476]:
query2=' SELECT PHU_NAME, RESOLVED_CASES, ACTIVE_CASES, FILE_DATE FROM cases WHERE RESOLVED_CASES > ACTIVE_CASES AND CAST(FILE_DATE AS STRING) LIKE "2020%"   '


In [477]:
df_q10 = sc.sql(query2)
df_q10.show(30,False)

+---------------------------------------+--------------+------------+---------+
|PHU_NAME                               |RESOLVED_CASES|ACTIVE_CASES|FILE_DATE|
+---------------------------------------+--------------+------------+---------+
|HALIBURTON, KAWARTHA, PINE RIDGE       |69            |23          |20200410 |
|HALTON REGION                          |137           |112         |20200410 |
|KINGSTON, FRONTENAC, LENNOX & ADDINGTON|39            |13          |20200410 |
|NIAGARA REGION                         |95            |91          |20200410 |
|NORTH BAY PARRY SOUND DISTRICT         |4             |2           |20200410 |
|NORTHWESTERN                           |8             |1           |20200410 |
|PETERBOROUGH COUNTY-CITY               |36            |14          |20200410 |
|RENFREW COUNTY AND DISTRICT            |6             |5           |20200410 |
|SUDBURY AND DISTRICT                   |19            |8           |20200410 |
|ALGOMA DISTRICT                        

# Question 11:

List only PHU which has least death due to covid-19 in each month.


In [528]:
from pyspark.sql import functions as F

df_csv=df_csv.withColumn("FILE_DATE", F.to_date(F.col("FILE_DATE").cast("string"), \
    'yyyyMMdd'))

In [530]:
df_csv.show(truncate=False)

+----------+---------------------------------------+-------+------------+--------------+------+
|FILE_DATE |PHU_NAME                               |PHU_NUM|ACTIVE_CASES|RESOLVED_CASES|DEATHS|
+----------+---------------------------------------+-------+------------+--------------+------+
|2020-04-10|HALIBURTON, KAWARTHA, PINE RIDGE       |2235   |23          |69            |18    |
|2020-04-10|HALTON REGION                          |2236   |112         |137           |9     |
|2020-04-10|HASTINGS & PRINCE EDWARD COUNTIES      |2238   |12          |11            |1     |
|2020-04-10|HURON PERTH                            |5183   |20          |8             |1     |
|2020-04-10|KINGSTON, FRONTENAC, LENNOX & ADDINGTON|2241   |13          |39            |0     |
|2020-04-10|LAMBTON COUNTY                         |2242   |65          |26            |9     |
|2020-04-10|LEEDS, GRENVILLE AND LANARK DISTRICT   |2243   |102         |36            |8     |
|2020-04-10|MIDDLESEX-LONDON            

In [569]:
query2='SELECT DISTINCT PHU_NAME,MONTH(FILE_DATE) AS MONTH, YEAR(FILE_DATE) AS YEAR  FROM cases WHERE DEATHS = (SELECT MIN(DEATHS) FROM cases) group by PHU_NAME, MONTH(FILE_DATE), YEAR(FILE_DATE)'

In [570]:
df_q11 = sc.sql(query2)
df_q11.show(truncate=False)

+---------------------------------------+-----+----+
|PHU_NAME                               |MONTH|YEAR|
+---------------------------------------+-----+----+
|HALDIMAND-NORFOLK                      |4    |2020|
|YORK REGION                            |4    |2020|
|KINGSTON, FRONTENAC, LENNOX & ADDINGTON|4    |2020|
|HALIBURTON, KAWARTHA, PINE RIDGE       |4    |2020|
|ALGOMA DISTRICT                        |5    |2020|
|GREY BRUCE                             |8    |2020|
|TIMISKAMING                            |7    |2020|
|EASTERN ONTARIO                        |5    |2020|
|DURHAM REGION                          |4    |2020|
|TIMISKAMING                            |4    |2020|
|RENFREW COUNTY AND DISTRICT            |4    |2020|
|TIMISKAMING                            |10   |2020|
|WINDSOR-ESSEX COUNTY                   |4    |2020|
|GREY BRUCE                             |11   |2020|
|BRANT COUNTY                           |4    |2020|
|HALTON REGION                          |4    

In [563]:
df_csv.show()

+----------+--------------------+-------+------------+--------------+------+
| FILE_DATE|            PHU_NAME|PHU_NUM|ACTIVE_CASES|RESOLVED_CASES|DEATHS|
+----------+--------------------+-------+------------+--------------+------+
|2020-04-10|HALIBURTON, KAWAR...|   2235|          23|            69|    18|
|2020-04-10|       HALTON REGION|   2236|         112|           137|     9|
|2020-04-10|HASTINGS & PRINCE...|   2238|          12|            11|     1|
|2020-04-10|         HURON PERTH|   5183|          20|             8|     1|
|2020-04-10|KINGSTON, FRONTEN...|   2241|          13|            39|     0|
|2020-04-10|      LAMBTON COUNTY|   2242|          65|            26|     9|
|2020-04-10|LEEDS, GRENVILLE ...|   2243|         102|            36|     8|
|2020-04-10|    MIDDLESEX-LONDON|   2244|         109|            53|     8|
|2020-04-10|      NIAGARA REGION|   2246|          91|            95|    12|
|2020-04-10|NORTH BAY PARRY S...|   2247|           2|             4|     0|

# Question 12:

Find out total active cases remaining in each PHU

In [323]:
query2='SELECT SUM(ACTIVE_CASES), PHU_NAME FROM cases GROUP BY PHU_NAME ORDER BY SUM(ACTIVE_CASES) ASC'

In [324]:
df_q12 = sc.sql(query2)
df_q12.show(30,False)

+-----------------+---------------------------------------+
|sum(ACTIVE_CASES)|PHU_NAME                               |
+-----------------+---------------------------------------+
|788              |TIMISKAMING                            |
|1185             |NORTH BAY PARRY SOUND DISTRICT         |
|1272             |ALGOMA DISTRICT                        |
|1633             |PORCUPINE                              |
|1777             |NORTHWESTERN                           |
|2155             |RENFREW COUNTY AND DISTRICT            |
|2856             |HASTINGS & PRINCE EDWARD COUNTIES      |
|3927             |SUDBURY AND DISTRICT                   |
|4082             |PETERBOROUGH COUNTY-CITY               |
|4680             |GREY BRUCE                             |
|5017             |KINGSTON, FRONTENAC, LENNOX & ADDINGTON|
|5311             |HALIBURTON, KAWARTHA, PINE RIDGE       |
|6730             |THUNDER BAY DISTRICT                   |
|6876             |LEEDS, GRENVILLE AND 

# Question 13:

Find out on which date highest and lowest death reported in “NIAGARA REGION” PHU.


In [353]:
query2='SELECT  FILE_DATE, DEATHS \
FROM cases \
WHERE DEATHS == (SELECT MIN(DEATHS) FROM cases) AND PHU_NAME=="NIAGARA REGION"'

In [354]:
df_q12 = sc.sql(query2)
df_q12.show(30,False)

+---------+------+
|FILE_DATE|DEATHS|
+---------+------+
|20200401 |0     |
+---------+------+



In [382]:
query2='SELECT FILE_DATE, MAX(DEATHS) \
FROM cases WHERE PHU_NAME=="NIAGARA REGION" GROUP BY FILE_DATE ORDER BY MAX(DEATHS) DESC'

In [383]:
df_q12 = sc.sql(query2)
df_q12.show(1,False)

+---------+-----------+
|FILE_DATE|max(DEATHS)|
+---------+-----------+
|20210124 |257        |
+---------+-----------+
only showing top 1 row



# Question 14:

Find out total resolved cases of covid-19 in “NORTH BAY PARRY SOUND DISTRICT” in month
May 2020 and Oct 2020

In [542]:
query2='SELECT DISTINCT PHU_NAME, SUM(RESOLVED_CASES) AS TOTAL_RESOLVED_CASES from cases where MONTH(FILE_DATE) IN (5,10) AND YEAR(FILE_DATE)=2020 group by PHU_NAME'


In [543]:
df_q14 = sc.sql(query2)
df_q14.show(30,False)

+---------------------------------------+--------------------+
|PHU_NAME                               |TOTAL_RESOLVED_CASES|
+---------------------------------------+--------------------+
|CITY OF HAMILTON                       |52560               |
|DURHAM REGION                          |95842               |
|KINGSTON, FRONTENAC, LENNOX & ADDINGTON|6562                |
|LEEDS, GRENVILLE AND LANARK DISTRICT   |18943               |
|THUNDER BAY DISTRICT                   |5543                |
|OXFORD ELGIN-ST.THOMAS                 |10182               |
|GREY BRUCE                             |6857                |
|TORONTO                                |749899              |
|LAMBTON COUNTY                         |15367               |
|NIAGARA REGION                         |48095               |
|HALTON REGION                          |57470               |
|SUDBURY AND DISTRICT                   |4964                |
|NORTHWESTERN                           |2457          

# Question 15:

Find out percentage of active cases in each PHU in year 2020.

In [539]:
#df_csv.select(to_date("FILE_DATE", "dd-MMM-yyyy").alias("date"))
#df_csv=df_csv.withColumn("FILE_DATE", f.to_date(CAST(FILE_DATE AS STRING(f.col("FILE_DATE")))))
query2='Select DISTINCT PHU_NAME, ROUND((SUM(ACTIVE_CASES)/(SUM(ACTIVE_CASES)+SUM(RESOLVED_CASES)))*100,2)  AS total_percentage FROM cases GROUP BY PHU_NAME'


In [531]:
df_csv.createOrReplaceTempView("cases")

In [532]:
#query2='select * from cases '

In [540]:
df_q12 = sc.sql(query2)
df_q12.show(30,False)

+---------------------------------------+----------------+
|PHU_NAME                               |total_percentage|
+---------------------------------------+----------------+
|CITY OF HAMILTON                       |12.92           |
|DURHAM REGION                          |9.29            |
|KINGSTON, FRONTENAC, LENNOX & ADDINGTON|9.2             |
|LEEDS, GRENVILLE AND LANARK DISTRICT   |6.3             |
|THUNDER BAY DISTRICT                   |12.46           |
|OXFORD ELGIN-ST.THOMAS                 |14.95           |
|GREY BRUCE                             |8.58            |
|TORONTO                                |10.89           |
|LAMBTON COUNTY                         |9.72            |
|NIAGARA REGION                         |13.3            |
|HALTON REGION                          |12.71           |
|SUDBURY AND DISTRICT                   |10.32           |
|NORTHWESTERN                           |8.78            |
|YORK REGION                            |10.63          

In [541]:
df_csv.show(20,False)

+----------+---------------------------------------+-------+------------+--------------+------+
|FILE_DATE |PHU_NAME                               |PHU_NUM|ACTIVE_CASES|RESOLVED_CASES|DEATHS|
+----------+---------------------------------------+-------+------------+--------------+------+
|2020-04-10|HALIBURTON, KAWARTHA, PINE RIDGE       |2235   |23          |69            |18    |
|2020-04-10|HALTON REGION                          |2236   |112         |137           |9     |
|2020-04-10|HASTINGS & PRINCE EDWARD COUNTIES      |2238   |12          |11            |1     |
|2020-04-10|HURON PERTH                            |5183   |20          |8             |1     |
|2020-04-10|KINGSTON, FRONTENAC, LENNOX & ADDINGTON|2241   |13          |39            |0     |
|2020-04-10|LAMBTON COUNTY                         |2242   |65          |26            |9     |
|2020-04-10|LEEDS, GRENVILLE AND LANARK DISTRICT   |2243   |102         |36            |8     |
|2020-04-10|MIDDLESEX-LONDON            